In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.optimizers import Adam
import matplotlib.pyplot as plt
from zipfile import ZipFile

In [3]:
with ZipFile("data/emotions.zip", "r") as _file:
    _file.extractall("data/emotions")

train_dir = "data/emotions/train"
test_dir = "data/emotions/test"

img_size = (48, 48)

In [4]:
train_datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    rescale=1.0 / 255,
    validation_split=0.2,
)
validation_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=img_size,
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical",
    subset="training",
)
validation_generator = validation_datagen.flow_from_directory(
    directory=test_dir,
    target_size=img_size,
    batch_size=64,
    color_mode="grayscale",
    class_mode="categorical",
    subset="validation",
)

In [ ]:
"""
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=(3, 3),  padding="same", activation="relu", input_shape=(*img_size, 1)))
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(BatchNormalization())

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(units=128, activation="relu", kernel_initializer="he_normal"))
model.add(Dropout(0.25))
model.add(Dense(units=64, activation="relu", kernel_initializer="he_normal"))

model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(units=32, activation="relu", kernel_initializer="he_normal"))
model.add(Dense(7, activation="softmax"))

model.compile(
    optimizer=Adam(lr=0.0001), loss="categorical_crossentropy", metrics=["accuracy"]
)
"""

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), padding="same", activation="relu", input_shape=(*img_size, 1)))
model.add(Conv2D(64, (3, 3), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (5, 5), padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, (3, 3), padding="same", activation="relu", kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(512, activation="relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(7, activation="softmax"))

model.compile(
    optimizer=Adam(lr=0.0001), loss="categorical_crossentropy", metrics=["accuracy"]
)

In [8]:
epochs = 60
model.summary()

In [ ]:
history = model.fit(
    x=train_generator, epochs=epochs, validation_data=validation_generator
)

In [ ]:
fig, ax = plt.subplots(1, 2)
train_acc = history.history["accuracy"]
train_loss = history.history["loss"]
fig.set_size_inches(12, 4)

ax[0].plot(history.history["accuracy"])
ax[0].plot(history.history["val_accuracy"])
ax[0].set_title("Training Accuracy vs Validation Accuracy")
ax[0].set_ylabel("Accuracy")
ax[0].set_xlabel("Epoch")
ax[0].legend(["Train", "Validation"], loc="upper left")

ax[1].plot(history.history["loss"])
ax[1].plot(history.history["val_loss"])
ax[1].set_title("Training Loss vs Validation Loss")
ax[1].set_ylabel("Loss")
ax[1].set_xlabel("Epoch")
ax[1].legend(["Train", "Validation"], loc="upper left")

plt.show()

In [12]:
model.save("data/model_emotions.h5")